<a href="https://colab.research.google.com/github/esterssc24/phising/blob/main/phishing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install pandas numpy scikit-learn joblib


In [13]:
# criar um arquivo csv
data = """url,label
http://example.com,0
http://phishing-site.com,1
https://secure-site.org,0
http://malicious-link.net,1
http://legit-site.com,0
http://fakebank-login.com,1
"""
with open("urls.csv", "w") as file:
    file.write(data)

print("arquivo urls.csv criado com sucesso!")


arquivo urls.csv criado com sucesso!


In [14]:
import requests
import joblib
from urllib.parse import urlparse
import numpy as np
import re

class Utils:
    def extract_features(self, url: str) -> np.array:
        """
        extrai características simples da url.
        """
        parsed_url = urlparse(url)
        return np.array([
            len(url),  # tamanho da url
            url.count('.'),  # número de pontos
            url.count('-'),  # número de hífens
            url.startswith('https'),  # começa com https
            len(parsed_url.netloc),  # tamanho do domínio
            len(parsed_url.path),  # tamanho do caminho
        ]).reshape(1, -1)

    def model_predict(self, classifier, features: np.array):
        """
        faz a predição usando o modelo.
        """
        try:
            probabilities = classifier.predict_proba(features)
            score = probabilities[0][1] * 100  # probabilidade de phishing
            reason = "alta chance de ser golpe" if score > 50 else "parece seguro"
            return {
                "empresa": "Desconhecida",
                "valida": True,
                "motivo": reason,
                "score": round(score, 2)
            }
        except Exception as e:
            print("erro na predição:", e)
            return {
                "empresa": "Desconhecida",
                "valida": False,
                "motivo": "erro ao tentar processar a predição"
            }

class Url:
    def __init__(self):
        self.classifier = None
        self.utils = Utils()

    def set_classifier(self, classifier):
        """
        define o classificador a ser usado.
        """
        self.classifier = classifier

    def is_valid_url(self, url: str) -> bool:
        """
        valida o formato da url.
        """
        regex = re.compile(
            r'^(https?:\/\/)?'  # protocolo
            r'([a-zA-Z0-9-]+\.)+[a-zA-Z]{2,6}'  # domínio
            r'(\/[a-zA-Z0-9._%+-]*)*$',  # caminho opcional
            re.IGNORECASE
        )
        return re.match(regex, url) is not None

    def check_phishing(self, url: str):
        """
        verifica se a url é phishing.
        """
        if not self.is_valid_url(url):
            return {
                "empresa": "Desconhecida",
                "valida": False,
                "motivo": "url inválida ou mal formatada"
            }

        # garantir que a url comece com http/https
        url = url if url.startswith('http') else ('http://' + url)

        # tentar acessar a url
        try:
            response = requests.get(url, timeout=5)
            response.raise_for_status()
        except requests.exceptions.RequestException as error:
            return {
                "empresa": "Desconhecida",
                "valida": False,
                "motivo": f"inválida, erro ao acessar a url ({error})"
            }

        # extrair características e fazer a predição
        try:
            features = self.utils.extract_features(url)
            result = self.utils.model_predict(self.classifier, features)
            return result
        except Exception as e:
            print("erro:", e)
            return {
                "empresa": "Desconhecida",
                "valida": False,
                "motivo": "erro ao tentar processar a predição"
            }


In [15]:
import joblib
import numpy as np
from urllib.parse import urlparse
import re

class UrlAnalyzer:
    def __init__(self):
        # carregue o modelo treinado
        try:
            self.model = joblib.load("phishing_model.pkl")
        except FileNotFoundError:
            self.model = None
            print("modelo não encontrado! você precisa treinar o modelo primeiro.")

    def is_valid_url(self, url: str) -> bool:
        """
        valida o formato da url usando regex.
        """
        regex = re.compile(
            r'^(https?:\/\/)?'  # protocolo
            r'([a-zA-Z0-9-]+\.)+[a-zA-Z]{2,6}'  # domínio
            r'(\/[a-zA-Z0-9._%+-]*)*$',  # caminho opcional
            re.IGNORECASE
        )
        return re.match(regex, url) is not None

    def extract_features(self, url: str) -> np.array:
        """
        extrai características simples da url para usar no modelo.
        """
        parsed_url = urlparse(url)
        return np.array([
            len(url),  # tamanho da url
            url.count('.'),  # número de pontos
            url.count('-'),  # número de hífens
            url.startswith('https'),  # começa com https
            len(parsed_url.netloc),  # tamanho do domínio
            len(parsed_url.path),  # tamanho do caminho
        ]).reshape(1, -1)

    def predict_phishing(self, url: str) -> dict:
        """
        classifica a url como phishing ou segura.
        """
        if not self.is_valid_url(url):
            return {
                "score": 100,
                "reason": "url inválida ou mal formatada"
            }

        if not self.model:
            return {
                "score": None,
                "reason": "modelo não está carregado. treine ou carregue um modelo para usar."
            }

        # extrair características e fazer a predição
        features = self.extract_features(url)
        score = self.model.predict_proba(features)[0][1] * 100  # probabilidade de ser phishing
        reason = "alta chance de ser golpe" if score > 50 else "parece seguro"

        return {
            "score": round(score, 2),
            "reason": reason
        }

# exemplo de uso
analyzer = UrlAnalyzer()

urls_to_test = [
    "http://example.com",
    "http://phishing-site.com",
    "https://secure-site.org",
    "http://fakebank-login.com"
]

for url in urls_to_test:
    result = analyzer.predict_phishing(url)
    print(f"url: {url} | score: {result['score']} | reason: {result['reason']}")


url: http://example.com | score: 11.0 | reason: parece seguro
url: http://phishing-site.com | score: 71.0 | reason: alta chance de ser golpe
url: https://secure-site.org | score: 5.0 | reason: parece seguro
url: http://fakebank-login.com | score: 94.0 | reason: alta chance de ser golpe
